<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [11]</a>'.</span>

## install libraries

In [1]:
!pip install -U unsloth


## Imports

In [2]:
from unsloth import FastLanguageModel
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch, random, os
from typing import Dict, List, Any
from transformers.data.data_collator import DefaultDataCollator
SEED = 42


/home/moein_salimi/users/babak/IdeaGeneration/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


INFO 09-06 01:29:10 [__init__.py:235] Automatically detected platform cuda.


🦥 Unsloth Zoo will now patch everything to make training faster!


## Load model and Tokenizer

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    "./Qwen3-4B-Instruct-2507",
    max_seq_length = 8192,
    dtype = torch.float16,
    load_in_4bit = True,
)

FastLanguageModel.for_training(model)
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=32,
    lora_dropout=0.0,
    use_rslora=True,
)


==((====))==  Unsloth 2025.9.1: Fast Qwen3 patching. Transformers: 4.53.3. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 4. Max memory: 23.559 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|                                                                                                                                  | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:  33%|████████████████████████████████████████▋                                                                                 | 1/3 [00:05<00:11,  5.62s/it]

Loading checkpoint shards:  67%|█████████████████████████████████████████████████████████████████████████████████▎                                        | 2/3 [00:12<00:06,  6.42s/it]

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  3.54s/it]

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.24s/it]

Unsloth 2025.9.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:

RESPONSE_TAG = "<|response|>"
tokenizer.add_special_tokens({"additional_special_tokens": [RESPONSE_TAG]})
model.resize_token_embeddings(len(tokenizer))
response_token_id = tokenizer.convert_tokens_to_ids(RESPONSE_TAG)

## Format function

In [5]:
SPECIAL_TOKENS = ["<|response|>", "<|analysis|>", "<|forecast|>"]
num_added = tokenizer.add_special_tokens({"additional_special_tokens": SPECIAL_TOKENS})
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))

RESPONSE_TAG = "<|response|>"
ANALYSIS_TAG = "<|analysis|>"
FORECAST_TAG = "<|forecast|>"

response_token_id = tokenizer.convert_tokens_to_ids(RESPONSE_TAG)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
tokenizer.padding_side     = "right"


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [6]:
import json


def normalize_output_to_commas(output: str) -> str:
    txt = str(output).strip()
    if txt.startswith("[") and txt.endswith("]"):
        try:
            arr = json.loads(txt)
            return ",".join(str(x).strip() for x in arr)
        except Exception:
            pass
    return ",".join([t.strip() for t in txt.split(",")])

def make_brief_analysis(thinking: str, limit_chars: int = 200) -> str:
    t = (thinking or "").strip()
    if not t:
        return "brief outlook based on the provided data"
    return t[:limit_chars].replace("\n", " ")

def format_chat(ex):
    instruction = ex.get("instruction", "") or ""
    user_input  = ex.get("input", "") or ""
    # thinking    = ex.get("custom_bitcoin_dataset", "") or ""
    output      = ex.get("output", "") or ""

    post_input_directive = (
        "Please analyze it first and then give me 10 next day prices separated by comma."
    )
    user_block = f"{user_input}\n\n{post_input_directive}"

    # brief_note = make_brief_analysis(thinking, limit_chars=20000)
    normalized_output = normalize_output_to_commas(output)

    assistant_payload = (
        f"{RESPONSE_TAG}\n"
        # f"{ANALYSIS_TAG} "
        f"{FORECAST_TAG}\n{normalized_output}"
    )

    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": user_block},
        {"role": "assistant", "content": assistant_payload},
    ]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

raw = load_dataset("tahamajs/Bitcoin-Long-Term-Trend-and-Price-Prediction-Dataset")
train_data = raw["train"].map(format_chat, remove_columns=raw["train"].column_names)

def tokenize_fn(ex):
    return tokenizer(ex["text"], truncation=True, max_length=4096, padding=False)

train_tok = train_data.map(tokenize_fn, batched=True, remove_columns=["text"])
train_tok = train_tok.shuffle(seed=SEED)



Repo card metadata block was not found. Setting CardData to empty.


In [7]:
import random
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

# --- Make sure these are defined in your main script ---
# You need the 'normalize_output_to_commas' function and the FORECAST_TAG constant
# def normalize_output_to_commas(output: str) -> str: ...
# FORECAST_TAG = "## Forecast"

class PrintRandomSampleCallback(TrainerCallback):
    def __init__(self, raw_dataset, tokenizer, num_steps=10):
        self.raw_dataset = raw_dataset
        self.tokenizer = tokenizer
        self.num_steps = num_steps

    def on_step_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        # Check if the current step is a multiple of `num_steps`
        if state.global_step > 0 and state.global_step % self.num_steps == 0:
            model = kwargs["model"]
            
            # 1. Pick a random sample from the raw dataset
            random_index = random.randint(0, len(self.raw_dataset) - 1)
            sample = self.raw_dataset[random_index]
            
            # 2. Reconstruct the user prompt based on the NEW dataset structure
            instruction = sample.get("instruction", "") or ""
            user_input  = sample.get("input", "") or ""
            post_input_directive = (
                "Please analyze it first and then give me 10 next day prices separated by comma."
            )
            user_block = f"{user_input}\n\n{post_input_directive}"

            prompt_messages = [
                {"role": "system", "content": instruction},
                {"role": "user", "content": user_block},
            ]
            prompt_for_model = self.tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )

            # 3. Reconstruct the ground truth answer for comparison
            output_prices = sample.get("output", "") or ""
            normalized_prices = normalize_output_to_commas(output_prices)
            ground_truth_response = f"{FORECAST_TAG}\n{normalized_prices}"
            
            # 4. Generate a response from the model
            inputs = self.tokenizer(prompt_for_model, return_tensors="pt").to(model.device)
            # Since the output is just prices, we don't need many new tokens
            outputs = model.generate(**inputs, max_new_tokens=2048, do_sample=True, top_p=0.9, temperature=0.7)
            
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Extract only the part generated by the assistant
            model_response = full_response.split(user_block)[-1].strip()
            
            # 5. Print a clear comparison
            print(f"\n" + "="*80)
            print(f"✅ CHECKPOINT AT STEP {state.global_step}")
            print(f" (Random Sample Index: {random_index})")
            print("-" * 80)
            print(f"▶️ PROMPT (Input part):\n{user_input[:600]}...\n")
            print(f"🤖 MODEL RESPONSE:\n{model_response}\n")
            print(f"🎯 GROUND TRUTH:\n{ground_truth_response}\n")
            print("="*80 + "\n")
collator_ = PrintRandomSampleCallback(raw["train"], tokenizer)

## Data callector

In [8]:
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from torch.nn.utils.rnn import pad_sequence
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.truncation_side = "left"
tokenizer.padding_side     = "right"

from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from torch.nn.utils.rnn import pad_sequence
import torch
from typing import Dict, List, Any

def _find_subsequence(haystack: torch.Tensor, needle: torch.Tensor) -> int:
    if needle.numel() == 0 or haystack.numel() < needle.numel():
        return -1
    for i in range(haystack.numel() - needle.numel() + 1):
        if torch.equal(haystack[i:i+needle.numel()], needle):
            return i
    return -1

class DataCollatorMaskResponse:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, response_token_id: int):
        self.tokenizer = tokenizer
        self.response_token_id = response_token_id

        assistant_start_str = tokenizer.apply_chat_template(
            [{"role":"assistant","content":""}],
            tokenize=False, add_generation_prompt=True
        )

        self.assistant_start_ids = torch.tensor(
            tokenizer(assistant_start_str, add_special_tokens=False)["input_ids"],
            dtype=torch.long
        )

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_ids_list      = [torch.tensor(f["input_ids"], dtype=torch.long) for f in features]
        attention_mask_list = [torch.tensor(f["attention_mask"], dtype=torch.long) for f in features]

        input_ids = pad_sequence(input_ids_list, batch_first=True,
                                 padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attention_mask_list, batch_first=True, padding_value=0)

        labels = input_ids.clone()

        for i in range(labels.size(0)):
            row = input_ids[i]

            pos = (row == self.response_token_id).nonzero(as_tuple=True)
            start_idx = -1
            if len(pos[0]) > 0:
                start_idx = int(pos[0][0].item())

            if start_idx < 0 and self.assistant_start_ids.numel() > 0:
                j = _find_subsequence(row, self.assistant_start_ids)
                if j >= 0:
                    start_idx = j + self.assistant_start_ids.numel() - 1

            if start_idx >= 0 and start_idx + 1 < row.numel():
                labels[i, : start_idx + 1] = -100
            else:
                keep = min(64, row.numel())
                labels[i, : row.numel() - keep] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }
collator = DataCollatorMaskResponse(tokenizer, response_token_id)



## Training Argumenst

In [9]:
args = TrainingArguments(
    output_dir="qwen_bitcoin_chat_fast_not_summerized_ttttt_",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=200,
    bf16=False,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    gradient_checkpointing=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,

    tokenizer=tokenizer,
    data_collator=collator,
    callbacks=[collator_], # Pass the callback in a list

)

/tmp/ipykernel_54415/2060009534.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,565 | Num Epochs = 4 | Total steps = 644
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 66,060,288 of 4,087,852,544 (1.62% trained)


Step,Training Loss
20,1.737400
40,1.366500
60,1.369300
80,1.361800
100,1.374500
120,1.381200
140,1.380800
160,1.731200
180,1.429700
200,1.374900


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Input IDs of shape torch.Size([1, 9230]) with length 9230 > the model's max sequence length of 8192.
We shall truncate it ourselves. It's imperative if you correct this issue first.



✅ CHECKPOINT AT STEP 10
 (Random Sample Index: 456)
--------------------------------------------------------------------------------
▶️ PROMPT (Input part):
[Technical Analysis]
Not fully available.

[Random Sample of News (last 60 days)]
DeFi Summer; Bitcoin Fall: The attention may have been with DeFi when it was warm, but as the cold winds of COVID-19 return fears and election volatility blow, bitcoin is resuming narrative dominance. For more episodes and free early access before our regular 3 p.m. Eastern time releases, subscribe with Apple Podcasts , Spotify , Pocketcasts , Google Podcasts , Castbox , Stitcher , RadioPublica , iHeartRadio or RSS . This episode is sponsored by Crypto.com , Bitstamp and Nexo.io . Today on the Brief: Coinbase off...

🤖 MODEL RESPONSE:
assistant
 best practices in financial services and compliance, and the financial infrastructure that supports them.
The U.S. has seen the most aggressive move in the past 10 years, with a new wave of regulation and enf

...


✅ CHECKPOINT AT STEP 620
 (Random Sample Index: 413)
--------------------------------------------------------------------------------
▶️ PROMPT (Input part):
[Technical Analysis for 2018-11-16]
BTC Price: 5575.55, BTC RSI: 16.88
Gold Price: 1220.80, Gold RSI: 52.89
Oil Price: 56.46, Oil RSI: 18.34

[Random Sample of News (last 60 days)]
Crypto Market Stable as Bitcoin Stabilizes at $6,350, Decentraland Surges 10%: bitcoin price us dollar Over the last 24 hours, the Bitcoin price has remained fairly stable in the $6,300 region with its volume stagnant at around $3.7 billion. Stellar (XLM) has increased by nearly six percent as the anticipation towards the potential listing of XLM by Coinbase, the world’s largest fiat-to-crypto exchange, continues to...

🤖 MODEL RESPONSE:
assistant
.35866434212149163652424917954259762291898.356319823464784.16627.6614127449829185255728338157544146276492315129519336883657.23537455357692871171267543737812196736748521987271516421.4172649422532.585.7553.5515


✅ CHECKPOINT AT STEP 630
 (Random Sample Index: 1556)
--------------------------------------------------------------------------------
▶️ PROMPT (Input part):
[Technical Analysis]
Not fully available.

[Random Sample of News (last 60 days)]
New York regulator issues final virtual currency rules: By Karen Freifeld and Gertrude Chavez-Dreyfuss NEW YORK (Reuters) - New York state issued on Wednesday extensive new rules for companies that operate in virtual currencies such as bitcoin but did little to accommodate complaints that overly tight regulation could hamper the nascent industry. The new rules, the first by a state, create comprehensive guidelines for regulating digital currency firms, according to the state's Department of Financial Services, wh...

🤖 MODEL RESPONSE:
assistant
.1.3.1.5.5.1.4.2.7.1.5.5..1.9.5.8..5.9.7.4.5.8.9.4.1.5.2.9.6.4.8.2.7..4..2.3..2.8.9.4.1..7.2.8.7.9.1.6.2.8.2.1.1.6.9.1.5.1.5.5.5.8.5.1.5.2.5.2.9.1.9.6.6.9.7.3.5.7.3.3.7

🎯 GROUND TRUTH:
<|forecast|>
Trend: u


✅ CHECKPOINT AT STEP 640
 (Random Sample Index: 1138)
--------------------------------------------------------------------------------
▶️ PROMPT (Input part):
[Technical Analysis for 2019-12-04]
BTC Price: 7252.03, BTC RSI: 35.52
Gold Price: 1474.00, Gold RSI: 50.71
Oil Price: 58.43, Oil RSI: 57.04

[Random Sample of News (last 60 days)]
Bitcoin, Ethereum & Ripple - American Wrap: 11/12/19: Bitcoin technical analysis: BTC/USD some hope as price moves within a bullish flag
• Bitcoin price is trading in positive territory, up some 1.30% in the second half of the session.
• BTC/USD price action is within consolidation mode, has been since breaching $9000.
• Critical near-term support eyed at $8500, failure to hold could be punishing.
Ethereum technical...

🤖 MODEL RESPONSE:
assistant
22859.74,137.96,4455235.18,371357.34,2983217.2.88.36.45.26.9.92.51.46.8.4.55.85.36.26.59.29.65.59.25.65.28.95.47.89.92.49.69.37.87.17.7.85.31.84.94.52.52.25.49.21.34.23.52.15.37.46.91.74.8.27.61.5.18.35.85.1

/home/moein_salimi/users/babak/IdeaGeneration/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:252: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=644, training_loss=1.719063699615668, metrics={'train_runtime': 27062.5449, 'train_samples_per_second': 0.379, 'train_steps_per_second': 0.024, 'total_flos': 9.328462483842662e+17, 'train_loss': 1.719063699615668, 'epoch': 4.0})

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [11]:
print("=== Sample formatted texts ===")
for i in range(2):
    print(train_data[i]["text"])
    print("="*80)

sample_batch = [train_to2k[i] for i in range(2)]
batch = collator(sample_batch)

print("\n=== Tokenized input_ids ===")
print(batch["input_ids"][0][:4000])
print("\nDecoded back:\n", tokenizer.decode(batch["input_ids"][0][:80]))

print("\n=== Labels ===")
print(batch["labels"][0][:4000])

masked_decoded = [
    tok if lab != -100 else "[MASK]"
    for tok, lab in zip(batch["input_ids"][0][:4000].tolist(), batch["labels"][0][:4000].tolist())
]
print("\nMasked Decoded (first 80 tokens):")
print(masked_decoded)


=== Sample formatted texts ===
<|im_start|>system
Analyze the provided historical prices, technical indicators, news, and social media sentiment from the last 60 days. Based on this context, predict the overall trend (up, down, or no change) and the specific daily closing prices for Bitcoin for the next 10 days.<|im_end|>
<|im_start|>user
[Technical Analysis for 2021-10-14]
BTC Price: 57321.52, BTC RSI: 68.34
Gold Price: 1796.70, Gold RSI: 60.27
Oil Price: 81.31, Oil RSI: 74.15

[Random Sample of News (last 60 days)]
Coinbase Expands Banking Services by Letting Users Deposit Paychecks Into Their Accounts: Coinbase has been in the news lately for getting on thewrong side of both customers and regulators, but the cryptocurrency exchange made a different kind of headline on Monday when it said users in the United States will now be able to deposit their paychecks directly into their online accounts.
See:Best Cryptocurrencies to Invest In for 2021Find:25 Best Startup Business Ideas to Make

NameError: name 'train_to2k' is not defined

## Save model

In [ ]:
trainer.model.save_pretrained("qwen_bitcoin_chat_fast/lora_adapter")
tokenizer.save_pretrained("qwen_bitcoin_chat_fast")
